In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import Input
 
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

Using TensorFlow backend.


In [2]:
input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer

'''Xception 架構'''
model = keras.applications.xception.Xception(include_top=False, weights='imagenet', 
                                             input_tensor=input_tensor, 
                                             pooling=None, classes=10)

'''Resnet 50 架構'''
#model=keras.applications.ResNet50(include_top=False, weights='imagenet',
                                    #input_tensor=input_tensor,
                                    #pooling=None, classes=10)
model.summary()

83689472/83683744 [==============================] - 2s 0us/step
Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 15, 15, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 15, 15, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 15, 15, 32)   0           block1_conv1_bn[0][0]            
__________________________

In [3]:
x = model.output
'''可以參考Cifar10實作章節'''
x = Convolution2D(32, (1,1) ,activation='relu')(x)
x = MaxPooling2D(pool_size=(1,1))(x)
x = Convolution2D(64, (1,1) ,activation='relu')(x)
x = Flatten()(x)
x = Dense(units=100,activation='relu')(x)

predictions = Dense(output_dim=10,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)
print('Model深度：', len(model.layers))

Model深度： 138


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`
  if __name__ == '__main__':


In [4]:

for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

In [5]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

170500096/170498071 [==============================] - 4s 0us/step
(50000, 32, 32, 3)


In [6]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size=32,epochs=30)

Epoch 1/30
50000/50000 [==============================] - 48s 958us/step - loss: 2.0878 - accuracy: 0.1853
Epoch 2/30
50000/50000 [==============================] - 39s 780us/step - loss: 1.2741 - accuracy: 0.5466
Epoch 3/30
50000/50000 [==============================] - 40s 791us/step - loss: 1.0498 - accuracy: 0.6404
Epoch 4/30
50000/50000 [==============================] - 39s 787us/step - loss: 0.9388 - accuracy: 0.6806
Epoch 5/30
50000/50000 [==============================] - 39s 788us/step - loss: 0.8511 - accuracy: 0.7093
Epoch 6/30
50000/50000 [==============================] - 40s 795us/step - loss: 0.7798 - accuracy: 0.7347
Epoch 7/30
50000/50000 [==============================] - 39s 788us/step - loss: 0.7108 - accuracy: 0.7577
Epoch 8/30
50000/50000 [==============================] - 40s 799us/step - loss: 0.6521 - accuracy: 0.7789
Epoch 9/30
50000/50000 [==============================] - 40s 790us/step - loss: 0.5967 - accuracy: 0.7986
Epoch 10/30
50000/50000 [============